# 03.4 Itô Integral and Itô Calculus
<h3><span style="color:#800000;"><strong>Authored by:</strong> <em>Alexandre Mathias DONNAT, Sr</em></span></h3>

**Goal of this notebook**
- Introduce the Itô integral $\int_0^t H_s \, dW_s$ starting from simple processes.
- Establish the Itô isometry:
$$E\left[\left(\int_0^t H_s \, dW_s\right)^2\right] = E\left[\int_0^t H_s^2 \, ds\right].$$
- Present Itô's formula in 1D and the multi-dimensional version.
- Validate Itô's formula numerically through simulations: ($d(W_t^2) = 2W_t \, dW_t + dt$, geometric Brownian motion and the distribution of $\ln S_t$)
- Illustrate numerical approximations of the Itô integral using Riemann–Itô sums.

**This notebook**

Builds the Itô integral step by step and prepares the transition to the final notebook on SDEs and numerical schemes.


# 1. From simple integrands to the Itô integral

### *Before to begin*

In continuous-time stochastic modelling, Brownian motion is used to represent the random shocks that drive asset prices, interest rates, and volatility. Models of the form

$$dX_t = b(t, X_t) \, dt + \sigma(t, X_t) \, dW_t$$

require a rigorous interpretation of the term $\sigma(t, X_t) \, dW_t$.

Classical Riemann or Lebesgue integrals cannot handle Brownian paths—these trajectories are nowhere differentiable and too irregular.

This difficulty was resolved in 1944 by the Japanese mathematician Kiyoshi Itô, who introduced the Itô integral and the foundations of what is now called Itô calculus. The construction begins with simple, piecewise-constant predictable integrands and extends to general square-integrable processes via an $L^2$-limit. The resulting integral has strong martingale properties and satisfies the fundamental Itô isometry.

In quantitative finance, Itô calculus is central: it underpins the Black–Scholes model, stochastic volatility models, interest-rate dynamics, and all SDEs where the term $\sigma \, dW_t$ represents instantaneous randomness. It also provides the mathematical machinery behind hedging strategies, risk-neutral valuation, and the derivation of pricing PDEs.

We work on a filtered probability space $(\Omega, \mathcal{F}, (\mathcal{F}_t), P)$ where $W_t$ is a standard Brownian motion.

## 1.1 Simple predictable processes

A process $H$ is simple predictable if it has the form

$$H_t = \sum_{k=0}^{n-1} H_k \mathbf{1}_{(t_k, t_{k+1}]}(t),$$

with $H_k$ measurable with respect to $\mathcal{F}_{t_k}$.

For such $H$, the Itô integral is defined by

$$\int_0^t H_s \, dW_s = \sum_{k: t_{k+1} \leq t} H_k (W_{t_{k+1}} - W_{t_k}).$$

This definition is consistent: increments are independent and Gaussian.

## 1.2 Extension to square-integrable processes

For a general predictable process $H$ satisfying

$$\mathbb{E}\left[\int_0^T H_s^2 \, ds\right] < \infty,$$

we approximate $H$ by simple processes and define the integral as an $L^2$-limit. This leads to the Itô isometry.


# 2. Itô isometry

**Theorem (Itô isometry).**  
For any predictable square-integrable process $H$,

$$\mathbb{E}\left[\left(\int_0^t H_s \, dW_s\right)^2\right] = \mathbb{E}\left[\int_0^t H_s^2 \, ds\right].$$

This identity is fundamental: the Itô integral behaves like an isometry from $L^2$-integrands to martingales.


# 3. Numerical approximation of an Itô integral

We approximate the integral by the left-point Riemann–Itô sum:

$$I^{(n)} = \sum_{k=0}^{n-1} H_{t_k} (W_{t_{k+1}} - W_{t_k}),$$

which converges in $L^2$ to $\int_0^T H_t \, dW_t$.

We start with $H_t = \sin(t)$.


The mean should be close to 0 and the variance close to the theoretical value.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

rng = np.random.default_rng(123)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True
# simulate Brownian increments for many paths
T = 1.0
n_steps = 2000
n_paths = 50_000

dt = T / n_steps
t_grid = np.linspace(0, T, n_steps+1)

# Brownian increments
dW = rng.normal(0.0, np.sqrt(dt), size=(n_paths, n_steps))
W = np.concatenate([np.zeros((n_paths,1)), np.cumsum(dW, axis=1)], axis=1)

# integrand H_t = sin(t)
H_vals = np.sin(t_grid[:-1])

# compute Riemann–Ito sums for all paths
Ito_approx = (H_vals * dW).sum(axis=1)

# theoretical variance = ∫ sin^2(t) dt = T/2 - sin(2T)/4
var_theoretical = T/2 - np.sin(2*T)/4


In [2]:
print("Empirical mean   ≈", Ito_approx.mean())
print("Empirical variance≈", Ito_approx.var(ddof=1))
print("Theoretical var  =", var_theoretical)

Empirical mean   ≈ 0.004159292839191021
Empirical variance≈ 0.2722662653865929
Theoretical var  = 0.2726756432935796


# 4. Itô's formula

Let $f \in C^2(\mathbb{R})$ and $X_t = W_t$.
Then the Itô formula states:

$$df(W_t) = f'(W_t) \, dW_t + \frac{1}{2} f''(W_t) \, dt.$$

Example:

$f(x) = x^2$ gives

$$d(W_t^2) = 2W_t \, dW_t + dt.$$

In multi-dimensional form, for a diffusion

$$dX_t = b(t, X_t) \, dt + \sigma(t, X_t) \, dW_t,$$

Itô's formula becomes

$$df(t, X_t) = \partial_t f \, dt + \sum_i \partial_{x_i} f \, dX_t^{(i)} + \frac{1}{2} \sum_{i,j} (\sigma \sigma^\top)_{ij} \, \partial_{x_i x_j}^2 f \, dt.$$


# 5. Numerical validation of Itô's formula for $W_t^2$

We check that

$$W_T^2 - W_0^2 - \int_0^T 2W_s \, dW_s \approx T.$$


The mean should be very close to 𝑇, confirming Itô’s formula numerically.

In [3]:
T = 1.0
n_steps = 2000
n_paths = 50_000

dt = T / n_steps
t_grid = np.linspace(0, T, n_steps+1)

# simulate Brownian motion
dW = rng.normal(0.0, np.sqrt(dt), size=(n_paths, n_steps))
W = np.concatenate([np.zeros((n_paths,1)), np.cumsum(dW, axis=1)], axis=1)

# Ito integral of 2W_t
H = 2 * W[:, :-1]     # left endpoint values
Ito_term = (H * dW).sum(axis=1)

lhs = W[:, -1]**2 - Ito_term  # should be close to T


In [4]:
print("Empirical mean of LHS ≈", lhs.mean())
print("Theoretical value     =", T)

Empirical mean of LHS ≈ 1.0000537794240643
Theoretical value     = 1.0


# 6. Itô formula for geometric Brownian motion (GBM)

The GBM solves

$$dS_t = \mu S_t \, dt + \sigma S_t \, dW_t, \quad S_0 > 0.$$

By Itô's formula applied to $f(x) = \ln x$,

$$d(\ln S_t) = \left(\mu - \frac{1}{2}\sigma^2\right) dt + \sigma \, dW_t,$$

hence

$$\ln S_t \sim N\left(\ln S_0 + \left(\mu - \frac{1}{2}\sigma^2\right)t, \, \sigma^2 t\right).$$

We now verify this numerically.


The empirical and theoretical quantities should match closely.

In [5]:
S0 = 1.0
mu = 0.3
sigma = 0.6
T = 1.0
n_paths = 50_000
n_steps = 2000

dt = T/n_steps
t_grid = np.linspace(0, T, n_steps+1)

dW = rng.normal(0.0, np.sqrt(dt), size=(n_paths, n_steps))
W = np.concatenate([np.zeros((n_paths,1)), np.cumsum(dW, axis=1)], axis=1)

# Euler-Maruyama for GBM (exact solution is known, but we test Ito)
S = np.zeros_like(W)
S[:,0] = S0
for k in range(n_steps):
    S[:, k+1] = S[:, k] + mu*S[:, k]*dt + sigma*S[:, k]*dW[:, k]

lnS = np.log(S[:, -1])

In [7]:
emp_mean = lnS.mean()
emp_var = lnS.var(ddof=1)

th_mean = np.log(S0) + (mu - 0.5*sigma**2)*T
th_var = sigma**2 * T

print("Empirical mean ≈", emp_mean)
print("Theoretical mean =", th_mean)
print()
print("Empirical var ≈", emp_var)
print("Theoretical var =", th_var)

Empirical mean ≈ 0.1172161094273386
Theoretical mean = 0.12

Empirical var ≈ 0.35946044205508293
Theoretical var = 0.36
